In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    KubernetesOnlineEndpoint,
    KubernetesOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential


In [2]:
# enter details of your AML workspace
subscription_id = "ea6d544d-2425-4667-8d38-51f050d9d69e"
resource_group = "rg-vtpoc-9dev"
workspace = "mlw-vtpoc-9dev"


In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)


In [4]:
# Define an endpoint name
endpoint_name = "my-endpoint"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=endpoint_name,
    description="this is a sample online endpoint with no auth",
    auth_mode=None,
)
print(endpoint_name)

endpt-05131022449319


In [5]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (endpt-05131022449319) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-05131022449319', 'description': 'this is a sample online endpoint with no auth', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/.azureml/inferencing/endpt-05131022449319'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x103abd350>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [6]:
model = Model(path="./models/vt-model/model.pkl")
env = Environment(
    conda_file="../../data-science/environment/train-conda.yml",
    image="crvtpoc9dev.azurecr.io/local_deploy_img:v1",
)

In [7]:
blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../../mlops/scripts", scoring_script="score.py"
    ),
    instance_count=1,
)

In [9]:

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (endpt-05131022449319 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM crvtpoc9dev.azurecr.io/local_deploy_img:v1
 ---> 847acc328956
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 24dbf85dd101
 ---> ff0ce05e095e
Step 3/6 : WORKDIR /var/azureml-app/
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in 6375da36253a
 ---> 3ae95359a350
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 457db22bef98
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
 ---> Running in b4faffa4d959
.Retrieving n

KubernetesOnlineDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-05131022449319', 'type': 'Kubernetes', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/misc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1225a1110>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '2fbc54b506f55723f0d238d0ea428147', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/misc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x12177ab50>, 'version': '1', 'latest_version': None, 'path': '/Users/amoh

In [10]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Failed', 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-05131022449319', 'description': 'this is a sample online endpoint with no auth', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/Users/amohajerani/Desktop/code/visit-t-poc/misc/local_deploy'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x121778b50>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [11]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=50
)

'2024-05-13 14:46:16,821 I [62] gunicorn.error - Using worker: sync\r\n2024-05-13 14:46:16,831 I [245] gunicorn.error - Booting worker with pid: 245\r\n/opt/miniconda/envs/inf-conda-env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dc_storage_enabled" has conflict with protected namespace "model_".\r\n\r\nYou may be able to resolve this warning by setting `model_config[\'protected_namespaces\'] = (\'settings_\',)`.\r\n  warnings.warn(\r\n2024-05-13 14:46:17,481 W [245] azmlinfsrv - Found extra keys in the config file that are not supported by the server.\r\nExtra keys = [\'AZUREML_ENTRY_SCRIPT\', \'AZUREML_MODEL_DIR\', \'HOSTNAME\']\r\n2024-05-13 14:46:18,670 I [245] azmlinfsrv - AML_FLASK_ONE_COMPATIBILITY is set. Patched Flask to ensure compatibility with Flask 1.\r\nInitializing logger\r\n2024-05-13 14:46:18,684 I [245] azmlinfsrv - Starting up app insights client\r\n<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)\r\n

In [33]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../../data/test-request.json",
    local=True,
)

'[30.32259750366211, 29.90568733215332]'

In [37]:
# let's deploy in the cloud . We should delete the reaming code later on
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "k8s-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="aks-dev",
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [38]:
ml_client.begin_create_or_update(endpoint).result()

KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://4.149.66.177/api/v1/endpoint/k8s-endpoint-02282157694833/score', 'openapi_uri': 'http://4.149.66.177/api/v1/endpoint/k8s-endpoint-02282157694833/swagger.json', 'name': 'k8s-endpoint-02282157694833', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/resourcegroups/rg-vtpoc-9dev/providers/microsoft.machinelearningservices/workspaces/mlw-vtpoc-9dev/onlineendpoints/k8s-endpoint-02282157694833', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:82335111-e642-4809-92c0-ac1e65471504:39a1ec80-aa93-468f-aa2a-68914faa422d?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/resourceGroups/rg-vt

In [39]:
from azure.ai.ml.entities._deployment.resource_requirements_settings import (
    ResourceRequirementsSettings,
)
from azure.ai.ml.entities._deployment.container_resource_settings import (
    ResourceSettings,
)


blue_deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../../mlops/scripts", scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    ),
)

In [40]:
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint k8s-endpoint-02282157694833 exists
Uploading scripts (0.03 MBs): 100%|██████████| 31152/31152 [00:00<00:00, 95593.53it/s]


Uploading model.pkl (< 1 MB): 100%|██████████| 190k/190k [00:00<00:00, 831kB/s]




.................................................................................................................................................................

KubernetesOnlineDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'k8s-endpoint-02282157694833', 'type': 'Kubernetes', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:82335111-e642-4809-92c0-ac1e65471504:51d94e60-ba41-47b6-9942-0bd6cf7be030?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/resourceGroups/rg-vtpoc-9dev/providers/Microsoft.MachineLearningServices/workspaces/mlw-vtpoc-9dev/onlineEndpoints/k8s-endpoint-02282157694833/deployments/blue', 'Resource__source_path': None, 'base_path': '/Users/amohajerani/Desktop/code/visit-t-poc/local_deploy', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x11cfc8410>, 'model': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69

In [41]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://4.149.66.177/api/v1/endpoint/k8s-endpoint-02282157694833/score', 'openapi_uri': 'http://4.149.66.177/api/v1/endpoint/k8s-endpoint-02282157694833/swagger.json', 'name': 'k8s-endpoint-02282157694833', 'description': 'this is a sample online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/resourcegroups/rg-vtpoc-9dev/providers/microsoft.machinelearningservices/workspaces/mlw-vtpoc-9dev/onlineendpoints/k8s-endpoint-02282157694833', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:82335111-e642-4809-92c0-ac1e65471504:ba9e0bd4-89cb-4425-9e0a-a9e29d6d448d?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ea6d544d-2425-4667-8d38-51f050d9d69e/resourceGroups/rg-vt

In [44]:


ml_client.online_endpoints.invoke(
   endpoint_name=online_endpoint_name,
   deployment_name='blue',
   request_file='../../data/test-request.json')

'[30.32259750366211, 29.90568733215332]'